In [1]:
from importlib import reload
import sys; sys.path.append("..")
import os
import persist_to_disk as ptd
ptd.config.set_project_path(os.path.abspath("../"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm

import _settings
import utils.utils as utils

%matplotlib inline

## Load the predictions

In [2]:
cache_path = './prediction_df.pkl'
if not os.path.isfile(cache_path):
    import pipeline.main as pm
    trained_key = 'MNISTCNN-MNISTSup-20230521_012403087576'
    predsdf = pm.read_prediction(trained_key, _settings.MNISTSup_NAME, split='test',  datakwargs={"sample_proba": 0.4, 'noise_level': None}, mode='loss')
    pd.to_pickle(predsdf, cache_path)
predsdf = pd.read_pickle(cache_path)
print(predsdf.columns)
# P0-9 are the probability predictions
# S0-9 are the logits
# Y0-9 are the labels
predsdf

Index(['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'S0', 'S1',
       'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'Y0', 'Y1', 'Y2', 'Y3',
       'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9'],
      dtype='object')


,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9
9906087|9903908,0.005944,0.085406,0.032050,0.014273,0.029247,0.989302,0.987261,0.016888,0.015648,0.015872,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9900967|9906563|9902012|9904143,0.008368,0.834940,0.058302,0.950887,0.999283,0.053460,0.105393,0.019422,0.060050,0.122298,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
9904323|9905826,0.003322,0.310086,0.059254,0.036744,0.102803,0.423366,0.967694,0.038244,0.043194,0.035128,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9905652|9908648|9903049|9909040|9902967|9903781,0.999991,0.432286,0.019061,0.999926,0.043912,0.893941,0.033945,0.923458,0.605714,0.574208,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
9902400,0.007377,0.016672,0.004697,0.040158,0.007688,0.827812,0.021843,0.108535,0.034455,0.065034,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907014|9907498,0.013292,0.924378,0.003893,0.100471,0.023037,0.573339,0.038366,0.044116,0.082862,0.035613,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9904859|9908550|9904637|9904793,0.054973,0.308444,0.831794,0.162792,0.058373,0.995889,0.314862,0.934931,0.071206,0.051888,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
9901411|9902459|9904222,0.998869,0.101023,0.970695,0.944789,0.039823,0.022844,0.002916,0.457095,0.032170,0.178776,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9903779|9907687|9900081|9903707|9909079,0.389844,0.126573,0.890799,0.090184,0.335186,0.282586,0.998941,0.168396,0.686591,0.864325,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


## Define the targets and set functions 

In [3]:
import data_utils as data_utils
reload(data_utils)

target_cost = 0.1
delta = 0.1

_Y = np.asarray([0,1,0,0,1,0,1,1,0,0]) # {1,4,6,7}
_S = np.asarray([1,1,0,0,1,0,0,1,0,1]) # predicted set {0,1,4,7,9}
_pred = _Y * 0.9 + 0.09

In [4]:
cost_fn = data_utils.AdditiveSetFunction(1., mode='cost')
cost_fn(_S, _Y) # 2 false positives, normalized to 0.2 (maximum is 10 false positves)

0.2

In [5]:
# This GeneralSetFunction 
weights = np.arange(10)
weights[0] = 10
val_fn = data_utils.AdditiveSetFunction(weights, mode='util')
val_fn(_S, _Y) # Corret predictions are {1,4,7} this gives 1 + 4 + 7 / sum([1,...,10]) = 12/55=0.2182

0.21818181818181817

In [6]:
proxy_fn = data_utils.AdditiveSetFunction(1., mode='proxy')
proxy_fn(S=_S, pred=_pred), ((1-_pred) * _S).sum()/10 # the sum of predicted error probability for {0,1,4,7,9}

(0.185, 0.185)

In [7]:
# The AdditiveSetFunction class implements the following method that allows for faster construction of prediction sets
# In this case, it sorts the utilility/proxy ratio and returns the sequence of nested sets
val_fn.greedy_maximize_seq(_pred, d_proxy = proxy_fn.values * (1-_pred))[0]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0]),
 array([0, 1, 0, 0, 1, 0, 1, 1, 0, 0]),
 array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0]),
 array([1, 1, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 0, 0, 1, 0, 1, 1, 1, 1]),
 array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]

## Define the FavMac object

In [8]:

import conformal.setlevel as cs
reload(cs)
calib_obj = cs.FavMac_GreedyRatio(cost_fn, val_fn, proxy_fn, target_cost=target_cost, delta=delta)

In [9]:
nclass = 10
logit = predsdf.reindex(columns=['S%d'%i for i in range(nclass)])
label = predsdf.reindex(columns=['Y%d'%i for i in range(nclass)])

In [10]:
# randomly permute the samples for experiment purposes
_perm_idx = logit.index[np.random.RandomState(42).permutation(len(logit))]
logit, label = logit.reindex(_perm_idx), label.reindex(_perm_idx)

In [11]:

def evaluate_online(obj, logits, labels, cost_fn, val_fn, 
                    burn_in=200, update=True):
    import datetime
    """
    obj: intialized (but not calibrated) cc.Calibrator 
    logits: np.ndarray of shape (n, K)
    labels: np.ndarray of shape (n, K)
    cost_fn: Callable, cost function
    val_fn: Callable, value function
    burn_in: burn_in period (used to calibrate but evaluated)
    """
    obj.init_calibrate(logits[:burn_in], labels[:burn_in])
    logger, log_stride = None, None
    start_time = datetime.datetime.now()
    
    ret = {"thres": [obj.t],  'cost': [None], 'value': [None], '|S|': [None]}
    for i, (_logit, _label) in tqdm.tqdm(enumerate(zip(logits[burn_in:], labels[burn_in:])), desc='evaluating', total=len(logits)-burn_in):
        predset, _ = obj(_logit, _label, update=update)
        ret['thres'].append(obj.t)
        ret['cost'].append(cost_fn(predset, _label))
        ret['value'].append(val_fn(predset, _label))
        ret['|S|'].append(predset.sum())
        if log_stride is not None and (i+1) % log_stride == 0:
            logger.info(f"|{i+1}: {datetime.datetime.now().strftime('%Y%m%d-%H%M%S-%f')}|")
    if log_stride is not None: logger.info(f"|{i+1}: {datetime.datetime.now().strftime('%Y%m%d-%H%M%S-%f')}|")
    assert max(ret['cost'][1:]) <= 1 and max(ret['value'][1:]) <= 1
    return pd.DataFrame(ret).iloc[1:]

In [12]:
result = evaluate_online(calib_obj, logit.values, label.values, cost_fn=cost_fn, val_fn=val_fn, burn_in=1000, update=True)
result

evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████| 8926/8926 [00:03<00:00, 2338.64it/s]


,thres,cost,value,|S|
1,1.002917,0.2,0.054545,3.0
2,1.002917,0.1,0.163636,3.0
3,1.002917,0.0,0.145455,1.0
4,1.002917,0.1,0.218182,3.0
5,1.002917,0.1,0.236364,3.0
...,...,...,...,...
8922,0.985854,0.0,0.290909,2.0
8923,0.985854,0.0,0.545455,4.0
8924,0.985854,0.1,0.181818,2.0
8925,0.985428,0.2,0.109091,4.0


In [13]:
print(f"Violation Frequency: {(result['cost'] > target_cost).mean():.3f} vs target: {delta:.3f}")

Violation Frequency: 0.102 vs target: 0.100


In [14]:
result.describe()

,thres,cost,value,|S|
count,8926.000000,8926.00000,8926.000000,8926.000000
mean,0.991978,0.07179,0.307233,3.592091
std,0.005687,0.06554,0.151200,1.322666
min,0.974077,0.00000,0.000000,1.000000
25%,0.986244,0.00000,0.200000,3.000000
50%,0.992143,0.10000,0.290909,3.000000
75%,0.995027,0.10000,0.400000,4.000000
max,1.005714,0.30000,0.981818,10.000000
